### <p style="text-align: right;"> &#9989; Put your name here.</p>

# Homework Assignment #2 (Individual)
## Modifying pre-existing projects, learning to dissect someone else's work, learning about chaos theory

Ian Malcolm explains Chaos theory (Jurassic Park)

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo("5cVLUPwrSmU",width=640,height=360)

### Goals for this homework assignment
By the end of this assignment, you should be able to:
* Digest another person's code and understand how it works. 
* Understand the difference between absolute and relative paths.
* Modify existing code to add additional functionality
* Explain a basic level of chaos theory to your friends at parties
* Generate output files and process them using utilities
* Compile a collection of images into a movie

### Assignment instructions

Work through the following assignment, making sure to follow all of the directions and answer all of the questions.

There are **25 points** possible on this assignment. Point values for each part are included in the section headers.

**This assignment is due at 11:59pm on Friday, February 8.** It should be uploaded into the "Homework Assignments" submission folder for Homework #2.  Submission instructions can be found at the end of the notebook.

### Using your git repository to track your progress on your assignment 

For this assignment, you're going to use the Git repository, "`cmse202-s19-turnin`", you created previously on GitHub so that you can track your progress on the assignment and preserve the final version that you turn in. In order to do this you need to:

1. In the repository, create a new directory called "`homework-02`".
2. Move this notebook into that new directory within your cloned repository.
3. Add the notebook file to your git repository with the appropriate git command.
4. Commit the addition to your repository using the appropriate git command.
5. Finally, to test that everything is working, "git push" the file so that it ends up in your GitHub repository.

If everything went as intended, the new folder and the notebook file should now show up on your GitHub account in the "`cmse202-s19-turnin`" repository you created for Homework 1.  Periodically, **you'll be asked to commit your changes to the repository and push them to the remote GitHub location**. Of course, you can always commit your changes more often than that, if you wish.  It can be good to get into a habit of committing your changes any time you make a significant modification, or when you stop working on the project for a bit.


---
---
## Part 1: The Double Pendulum

A double pendulum is a pendulum attached to the end of another pendulum. 
It is a popular example that demonstrates what is called chaos:

https://en.wikipedia.org/wiki/Chaos_theory

The equations of motion of the double pendulum system are surprisingly complicated and are often solved numerically. 

### Part 1.1 Digesting Someone else's code (3 points)

We have provided some code from a sci-python example found at

https://scipython.com/blog/the-double-pendulum/

The code sets up the pendulum problem as such:
<img src="https://scipython.com/static/media/uploads/blog/double_pendulum/double-pendulum-geometry.svg" width=450><p style="text-align: right;">
</p>
The code solves for the time-evolution of $\theta_1$ and $\theta_2$, which ends up looking like the following:
<img src="https://scipython.com/static/media/uploads/blog/double_pendulum/double_pendulum.gif" width=450><p style="text-align: right;">
</p>





**Do this**: Run the following cell.  You will get an error.  Add code that uses the `!` operator and some CLI commands to create the missing directory. Now try to run the code again.  

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import pendulum_functions

theta1_initial = 3*np.pi/7 #initial conditions
dtheta1dt_initial = 0
theta2_initial =  3*np.pi/4
dtheta2dt_initial = 0

# Maximum time, time point spacings and the time grid (all in s).
tmax, dt = 30, 0.01
t = np.arange(0, tmax+dt, dt)
# Initial conditions: theta1, dtheta1/dt, theta2, dtheta2/dt.
y0 = np.array([theta1_initial, dtheta1dt_initial, theta2_initial , dtheta2dt_initial])

# Do the numerical integration of the equations of motion
y = odeint(pendulum_functions.deriv, y0, t )

# Check that the calculation conserves total energy to within some tolerance.
EDRIFT = 0.05

# Unpack z and theta as a function of time
theta1, theta2 = y[:,0], y[:,2]

# Convert to Cartesian coordinates of the two bob positions.
x1 = pendulum_functions.L1 * np.sin(theta1)
y1 = -pendulum_functions.L1 * np.cos(theta1)
x2 = x1 + pendulum_functions.L2 * np.sin(theta2)
y2 = y1 - pendulum_functions.L2 * np.cos(theta2)

# Plotted bob circle radius
r = 0.05
# Plot a trail of the m2 bob's position for the last trail_secs seconds.
trail_secs = 1
# This corresponds to max_trail time points.
max_trail = int(trail_secs / dt)

def make_plot(i):
    # Plot and save an image of the double pendulum configuration for time
    # point i.
    # The pendulum rods.
    ax.plot([0, x1[i], x2[i]], [0, y1[i], y2[i]], lw=2, c='k')
    # Circles representing the anchor point of rod 1, and bobs 1 and 2.
    c0 = Circle((0, 0), r/2, fc='k', zorder=10)
    c1 = Circle((x1[i], y1[i]), r, fc='r', ec='r', zorder=10)
    c2 = Circle((x2[i], y2[i]), r, fc='r', ec='r', zorder=10)
    ax.add_patch(c0)
    ax.add_patch(c1)
    ax.add_patch(c2)

    # The trail will be divided into ns segments and plotted as a fading line.
    ns = 20
    s = max_trail // ns

    for j in range(ns):
        imin = i - (ns-j)*s
        if imin < 0:
            continue
        imax = imin + s + 1
        # The fading looks better if we square the fractional length along the
        # trail.
        alpha = (j/ns)**2
        ax.plot(x2[imin:imax], y2[imin:imax], c='r', solid_capstyle='butt',
                lw=2, alpha=alpha)

    # Centre the image on the fixed anchor point, and ensure the axes are equal
    ax.set_xlim(-pendulum_functions.L1-pendulum_functions.L2-r, pendulum_functions.L1+pendulum_functions.L2+r)
    ax.set_ylim(-pendulum_functions.L1-pendulum_functions.L2-r, pendulum_functions.L1+pendulum_functions.L2+r)
    ax.set_aspect('equal', adjustable='box')
    plt.axis('off')
    plt.savefig('frames/_img{:04d}.png'.format(i//di), dpi=72)
    plt.cla()


# Make an image every di time points, corresponding to a frame rate of fps
# frames per second.
# Frame rate, s-1
fps = 10
di = int(1/fps/dt)
fig = plt.figure(figsize=(8.3333, 6.25), dpi=72)
ax = fig.add_subplot(111)

for i in range(0, t.size, di):
    print(i // di, '/', t.size // di)
    make_plot(i)

What did it do?  What files did it create? Where are those files?

<font size=8 color="#009600">&#9998;</font> Do This - Erase the contents of this cell and replace it with your answer to the above question!  (double-click on this text to edit this cell, and hit shift+enter to save the text). Please use appropriate Markup formating to make your answer's easy to read.


### Important note about creating the directory:
An absolute path is one that starts at the topmost directory (such as “C:” or “username:”) and lists a path of all folders to the desired target. Some examples of absolute paths are:

/Users/my_user_name/the_name_of_the_class_I_am_taking/homework_02/

OR

C:/Documents/my_user_name/the_name_of_the_class_I_am_taking/homework_02/

You’ll notice that this is the path you see when you use the pwd command. Say you wanted to share your code with a colleague. Let assume the code you are running is in a directory called “code” and calls a file in a directory called “code_subfolder”.  If you write your code such that it calls the file in “code_subfolder”  using an absolute path then the code probably won’t run on any computer except yours.  This is because when your code uses the path

/Users/jane_smith/cmse202/code/code_subfolder

The correct path on someone else’s computer might be

/Users/john_smith/cmse202/code/code_subfolder

The solution to this problem is to use relative paths, such as

./code_subfolder

Essentially, the period is replaced with with the working directory from where the code is executed, which from the command line is the equivalent to typing in the result of the pwd command.   This is useful for two reasons: (1) we don’t have to type out a long path to the correct folder and (2) this gives our code “portability”.  That is, the code can be run on anyone’s computer.  If you send us code that relies on an absolute path, then we cannot run the code and thus cannot assess your work.


### Another important note about making the directories:

If you manually make directories outside of the Jupyter notebook, then those directories will not exist when the TA runs your notebook to grade it. If your homework submission does not create the needed directories within the notebook,  your code will produce an error. This can be fixed by recalling how to use the “!” operator along with the "mkdir" command.

### Important reminder about turning assignments and exams in:

The TA grades your homework by using the “Restart and run all” option in the notebook.  Make sure your code runs all the way through.  The best way to make sure your code will work, is to copy your notebook (and the supporting function file) to another location and run the code via “restart and run all”.  Your code should produce all outputs with no errors when run in this manner.

**Do this**: Given what you just learned about relative paths and turning your assignments in to the TA, double check that the code you wrote will work when the TA tries to run it from their computer. 

### Part 1.2 Using utilities to post-process data (4 points)

The massive collection of images generated shows us the trajectory of the pendulum. However, it would be very useful if this was animated.  Try to run the following cell. It probably won't work. Why not? How do we fix this?

Note: You are welcome to find a different solution for animating a collection of images into a single animated gif, such as the package ``imageio`` or matplotlib's animation functionality.  Don't forget to review recent assignments for information on how to install software. 

In [ ]:
!convert frames/*.png solution_single.gif

Now we want to display the animation we just created. Debug the following code to display your image in this notebook.

<img src="./thebugisthatiamusingthewrongfilename.gif" width=450><p style="text-align: right;">
</p>

### Part 1.3 Cleaning up Extra Files Generated During Running Code (2 points)

Sometimes when we write code we can generate a great deal of files or functions that only get used during part of the code execution but are not otherwise useful. To free up memory or hard drive space, we should delete these files when we are done with them. The computer science community calls this "garbage collection". 

Find a method for deleting the images that you generated above, but NOT the animated gif. Remember that your method for deleting these files should work on other computers.  Namely, when the TA runs your code to grade it, the code should work for them. 

In [ ]:
# Write your code here


---
### STOP
Take a moment to save your notebook, commit the changes to your Git repository using the commit message "Finished Part 1", and push the changes to GitHub.

---


## Part 2 
Let's explore a situation in which chaos does not play a big role. We want to take our triple pendulum code and see what happens when the initial deflection angles are small.  For small deflection angles, a physicist will tell you that the equations for the double pendulum system end up looking like a simple spring, which is not a chaotic system.  A mathematician will tell you that you are in the "linear regime" of a chaotic system, ie where the equations are much simpler. 

### Part 2.1 Exploring non-chaotic regimes of a chaotic system (6 points)

We now want you to modify the code to model 3 pendulums simultaneously (the pendulums don't interact, we just want to superimpose their trajectories).  The new code must do the following:

* Display three double-pendulums simultaneously evolving in time, with their trajectories superimposed on one another. The initial conditions must be different, otherwise you will only see three identical pendulums on top of one another. 
* Modify the code to produce the system with the following initial conditions, plotting the pendulums with different colors. 
* Double Pendulum 1 (green) will begin at $\theta_1 = 3\pi/64$ and $\theta_2 = 3\pi/32$
* Double Pendulum 2 (blue) will begin at $\theta_1 = 3\pi/64*.9$ and $\theta_2 = 3\pi/32*.9$
* Double Pendulum 3 (red) will begin at $\theta_1 = 3\pi/64*1.1$ and $\theta_2 = 3\pi/32*1.1$
* The results should be compiled into an animated gif called `solution_triple_linear.gif`
* The image data should be deleted once you have finished creating the gif.
* Answer the following questions. 

Before writing the new code, outline your plan of attack for modifying this relatively large code.  

<font size=8 color="#009600">&#9998;</font> Do This - Erase the contents of this cell and replace it with your answer to the above question!  (double-click on this text to edit this cell, and hit shift+enter to save the text). Please use appropriate Markup formating to make your answer's easy to read.

At the start of the simulation, do the three pendulums start off close together?

<font size=8 color="#009600">&#9998;</font> Do This - Erase the contents of this cell and replace it with your answer to the above question!  (double-click on this text to edit this cell, and hit shift+enter to save the text). Please use appropriate Markup formating to make your answer's easy to read.

By the end of the simulation, are the pendulums following similar trajectories compared to one another?

<font size=8 color="#009600">&#9998;</font> Do This - Erase the contents of this cell and replace it with your answer to the above question!  (double-click on this text to edit this cell, and hit shift+enter to save the text). Please use appropriate Markup formating to make your answer's easy to read.

In [ ]:
### Write your code here


In [ ]:
### Write your code to make the gif here

Write your code to display the gif here.  Notice how this type of cell should be "markdown".  You can change this by going to "Cell"->"Cell Type" in the above drop down menu. 

### Part 2.2 Garbage Collection (2 points)

Perform garbage collection for this portion of the code, just as you did above, only using the space below.  Remember: the command must work when the TA runs this code on their computer.

In [ ]:
## Write your code here


---
### STOP
Take a moment to save your notebook, commit the changes to your Git repository using the commit message "Finished Part 2", and push the changes to GitHub.

---

## Part 3 Exploring chaotic regimes of a chaotic system

Intuition tells us that if three identical objects in a system start off in similar (but not identicaly) initial states, then they should behave mostly the same way.  We saw this happen in Part 2. However, objects starting at similar initial states in a chaotic systems can have wildly different behavior as time goes on. In this part of the homework we are going to explore this idea.  We will again model three pendulums that start near each other, but the initial conditions are different from above. 


### Part 3.1 (6 points)

We now want you to modify the code to model 3 pendulums simultaneously (the pendulums don't interact, we just want to superimpose their trajectories).  

Let's say (for example) that we are concernd about whether or not our users are colorblind. In this part of the homework, let's have the pendulum markers be three different shapes.

The new code must do the following:

* Display three double-pendulums simultaneously evolving in time, just as above.
* Double Pendulum 1 (green squares), which will be plotted with green circles, will begin at $\theta_1 = 3\pi/7$ and $\theta_2 = 3\pi/4$
* Double Pendulum 2 (blue circles), which will be plotted with blue circles, will begin at $\theta_1 = 3\pi/7*.9$ and $\theta_2 = 3\pi/4*.9$ (each deflection angle decreased by 10%)
* Double Pendulum 3 (red triangles), which will be plotted with red circles, will begin at $\theta_1 = 3\pi/7*1.1$ and $\theta_2 = 3\pi/4*1.1$ (each deflection angle increased by 10%)
* The results should be compiled into an animated gif called `solution_triple.gif`, which should be displayed in the notebook.
* Answer the following questions. 

Hint: How are the circles made?  There are similar commands for rectangles and wedges, which will be what you need for this part of the assignment. 

At the start of the simulation, do the three pendulums start off close together?

<font size=8 color="#009600">&#9998;</font> Do This - Erase the contents of this cell and replace it with your answer to the above question!  (double-click on this text to edit this cell, and hit shift+enter to save the text). Please use appropriate Markup formating to make your answer's easy to read.

By the end of the simulation, are the pendulums following similar trajectories compared to one another?

<font size=8 color="#009600">&#9998;</font> Do This - Erase the contents of this cell and replace it with your answer to the above question!  (double-click on this text to edit this cell, and hit shift+enter to save the text). Please use appropriate Markup formating to make your answer's easy to read.

In [ ]:
## Write your code here


In [ ]:
## Write your code to make the gif here

Write your code to display the gif here

### Part 3.2 Garbage Collection (2 points)

Perform garbage collection for this portion of the code.  Remember, the code you use to do this must work on the TA's computer when they grade your assignment. 

In [ ]:
## Write your code here

### Part 3.3 Code Portability

All your code in this notebook should work when the TA runs in on their computer.  The TA will use the "Restart and Run All" command, so be sure that your code still runs when you use this command. 


---
### STOP
Take a moment to save your notebook, commit the changes to your Git repository using the commit message "Finished Part 3", and push the changes to GitHub.

---

---
### Assignment wrap-up

Please fill out the form that appears when you run the code below.  **You must completely fill this out in order to receive credit for the assignment!**

Upload all three gif animations to the dropbox on D2L, along with your modified notebook. 

In [ ]:
from IPython.display import HTML
HTML(
"""
<iframe 
	src="https://goo.gl/forms/6JeCqg9FAsIPr4U53" 
	width="800px" 
	height="600px" 
	frameborder="0" 
	marginheight="0" 
	marginwidth="0">
	Loading...
</iframe>
"""
)

### Congratulations, you're done!

Submit this assignment by uploading it to the course Desire2Learn web page.  Go to the "Homework Assignments" folder, find the submission link for Homework #2, and upload it there.

&#169; Copyright 2018,  Michigan State University Board of Trustees